In [1]:
import warnings

import ray
from ray.rllib.utils import PolynomialSchedule

from src.components.train import train_env
from src.simglucose.env import register_simglucose_env

warnings.filterwarnings('ignore')
ray.init(log_to_driver=False)

/home/hamza/PycharmProjects/Autonomous-Insulin-Infusion-RL/venv/lib/python3.10/site-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if (distutils.version.LooseVersion(tf.__version__) <
2023-05-20 18:18:57,529	INFO worker.py:1538 -- Started a local Ray instance.


Python version:,3.10.6
Ray version:,2.2.0


# Define Configs

In [2]:
from src.simglucose.rewards import tan_reward, uniform_reward_with_risk

schedule_timesteps = 2000000
pl_sch = PolynomialSchedule(schedule_timesteps=2000000, initial_p=1e-3, final_p=1e-4, framework="torch", power=1)
lr_schedule = list(map(lambda t: [t, pl_sch.value(t)], range(0, schedule_timesteps, 2000)))
entropy_pl_sch = PolynomialSchedule(schedule_timesteps=2000000, initial_p=1e-3, final_p=1e-6, framework="torch",
                                    power=3)
ent_schedule = list(map(lambda t: [t, entropy_pl_sch.value(t)], range(0, schedule_timesteps, 2000)))
total_workers = 10
num_envs_per_worker = 1

env_name = "Simglucose-v0"
register_simglucose_env(env_name)
env_configs = dict(reward_fun=uniform_reward_with_risk, patient_name='adult#004')

In [3]:
from ray.rllib.algorithms.ppo import PPOConfig

lstm_model = {"fcnet_hiddens": [32, 32, 32], "vf_share_layers": False, "use_lstm": True,
              "lstm_cell_size": 32, "max_seq_len": 100}
algo = "PPO"
config = (
    PPOConfig()
    .environment(env_name, env_config=env_configs)
    .training(gamma=0.996, num_sgd_iter=3, sgd_minibatch_size=400, clip_param=0.1, lr=1e-3,
              train_batch_size=4000,
              entropy_coeff=1e-3, entropy_coeff_schedule=ent_schedule, lr_schedule=lr_schedule, vf_clip_param=10000)
    .resources(num_gpus=1, num_cpus_per_worker=1)
    .rollouts(num_rollout_workers=total_workers, num_envs_per_worker=num_envs_per_worker, enable_connectors=True, observation_filter='MeanStdFilter', batch_mode='complete_episodes')
    .framework("torch")
    .training(
        model=lstm_model)
    .evaluation(evaluation_num_workers=1)
)


In [4]:
# from ray.rllib.algorithms.dreamer import DreamerConfig
#
# config = (DreamerConfig().
#           training(gamma=0.996)
#
#           .resources(num_gpus=1, num_cpus_per_worker=1)
#           .rollouts(num_rollout_workers=total_workers, num_envs_per_worker=num_envs_per_worker)
#           .framework("torch")
#           .training(
#     model=lstm_model)
#           .evaluation(evaluation_num_workers=1)
#           )

In [5]:
log_dir = "tmp/pipeline_logs"

# Train RL Agent

The model is trained here using the best config from the tune step. The best training checkpoint is then chosen for evaluation

In [ ]:
train_results = train_env(
    algo=algo,
    config=config,
    log_dir=log_dir,
    iterations=20000,
    stop_reward_mean=1000,
    name="simglucose_solver",
    checkpoint_frequency=5
)
best_checkpoint = train_results.get_best_result(metric="episode_reward_mean", mode="max").best_checkpoints[0]
best_checkpoint_path = best_checkpoint[0]._local_path

2023-05-20 18:18:59,123	WARNING algorithm_config.py:571 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.
2023-05-20 18:18:59,126	WARNING algorithm_config.py:571 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.


Trial name,agent_timesteps_total,counters,custom_metrics,date,done,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,episodes_total,experiment_id,hostname,info,iterations_since_restore,node_ip,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_trained,num_env_steps_trained_this_iter,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,pid,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,time_since_restore,time_this_iter_s,time_total_s,timers,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
PPO_Simglucose-v0_06a8c_00000,4083,"{'num_env_steps_sampled': 4083, 'num_env_steps_trained': 4083, 'num_agent_steps_sampled': 4083, 'num_agent_steps_trained': 4083}",{},2023-05-20_18-19-26,False,16.9419,{},2.12176,1.19943,0.28526,241,241,a15cf10c789845628eb6cabd75b42f14,hamza-Legion-5-15ACH6H,"{'learner': {'default_policy': {'custom_metrics': {}, 'learner_stats': {'cur_kl_coeff': 0.20000000000000007, 'cur_lr': 0.0010000000000000002, 'total_loss': 0.13348971574256818, 'policy_loss': 0.006307779659982771, 'vf_loss': 0.12796844815214475, 'vf_explained_var': 0.3390824536482493, 'kl': 0.002509225061900736, 'entropy': 1.28835479815801, 'entropy_coeff': 0.0010000000000000002}, 'model': {}, 'num_grad_updates_lifetime': 15.5, 'diff_num_grad_updates_vs_sampler_policy': 14.5}}, 'num_env_steps_sampled': 4083, 'num_env_steps_trained': 4083, 'num_agent_steps_sampled': 4083, 'num_agent_steps_trained': 4083}",1,192.168.0.185,4083,4083,4083,4083,4083,4083,0,10,0,0,4083,"{'cpu_util_percent': 64.35882352941178, 'ram_util_percent': 88.56470588235294}",148076,{},{},{},"{'mean_raw_obs_processing_ms': 1.765510754995826, 'mean_inference_ms': 1.5396314379706169, 'mean_action_processing_ms': 0.193541997606311, 'mean_env_wait_ms': 20.45460580531374, 'mean_env_render_ms': 0.0}",11.4144,11.4144,11.4144,"{'training_iteration_time_ms': 11394.644, 'load_time_ms': 15.98, 'load_throughput': 255506.799, 'learn_time_ms': 306.226, 'learn_throughput': 13333.289, 'synch_weights_time_ms': 3.629}",1684599566,0,4083,1,06a8c_00000,11.8756
